# Prepare Dataset

## Helper function

getLaterDx: get the diagnosis after 6,12,18 months. 
 - input: 
   - data: the whole dataset table.
   - x :   a table of two column, "RID" and "M"
           which identifying the patient ID and months since base line.
   - months_specify: number of month after the diagnosis.
 - return: "NA" if the diagnosis not found.
           or a factor of later diagnosis.

In [1]:
"""
*CHANGE: 
Here I get the later diagnosis of all entries and mutate the value to a new column. 
I'll change the structure here to only include baseline, m06, 012, 018 entries.
"""

getLaterDx <- function(data, x, months_specify){
  RID_in <- as.integer(x["RID"])
  M_in <- as.integer(x["M"])
  if(!any(data$RID==RID_in&data$M==as.integer(M_in+months_specify))){
    return(NA)
  }
  return(as.factor(data[data$RID==RID_in&data$M==as.integer(M_in+months_specify),]$DX))
  
}

In [2]:
library(nycflights13)
library(tidyverse)
library(rstan)
library(ADNIMERGE)
library(caret)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: StanHeaders
rstan (Version 2.19.2, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)

Attaching package: ‘rstan’

The following object is masked from ‘package:tidyr’:

    extract

Loading required package: Hmisc
Loading required package: lattice
Loading required package: survival
Loading required package: Formula

Attaching package: ‘Hmisc’

The following objects are masked from ‘package:

In [3]:
adnimerge <- ADNIMERGE::adnimerge 
freesurfer51 <- ADNIMERGE::ucsffsl51all # 2010 entries, freesurfer 5.1
freesurfer44 <- ADNIMERGE::ucsffsl # 3572 entries, freesurfer 4.4
dim(freesurfer51)
dim(freesurfer44)

[1] 2010  363

[1] 3572  374

adnimerge_table contains `APEO4`, `Education`, `sex`, `ADAS`, `MMSE`, `CDR-SB`, `RAVLT- immidiate recall`

freesurfer table contains several columns with all NA entries. range :
  - freesurfer51: col 21 - 363
  - freesurfer44: col 29 - 374
  

## dataset filtering

### processing freesurfer table

In [4]:
"""
Here I make a summary on those columns with too many NAs (probably all NAs)
"""

remove_index_col <- c()
for (col in 29:374){
    if(any(is.na(freesurfer44[,col]))){
        
        print(col)
        print(sum(is.na(freesurfer44[,col])))
        print("===")
        remove_index_col <- append(remove_index_col, col)
    }
}

[1] 29
[1] 3572
[1] "==="
[1] 107
[1] 3572
[1] "==="
[1] 113
[1] 47
[1] "==="
[1] 114
[1] 3572
[1] "==="
[1] 145
[1] 3572
[1] "==="
[1] 146
[1] 3572
[1] "==="
[1] 147
[1] 3572
[1] "==="
[1] 148
[1] 3572
[1] "==="
[1] 166
[1] 3572
[1] "==="
[1] 178
[1] 3572
[1] "==="
[1] 205
[1] 3572
[1] "==="
[1] 283
[1] 3572
[1] "==="
[1] 289
[1] 10
[1] "==="
[1] 290
[1] 3572
[1] "==="
[1] 315
[1] 3572
[1] "==="
[1] 316
[1] 3572
[1] "==="
[1] 317
[1] 3572
[1] "==="
[1] 318
[1] 3572
[1] "==="
[1] 336
[1] 3572
[1] "==="
[1] 338
[1] 1
[1] "==="
[1] 339
[1] 1660
[1] "==="
[1] 348
[1] 3572
[1] "==="


should remove these columns(except 338), since they are all NAs.

For column 338, there is only one NA. should consider remove that row instead.

Later I'll do a NA filter to exclude those rows with NA values.

In [5]:
remove_index_col <- remove_index_col[!remove_index_col %in% 338] # exclude column 338

In [6]:
freesurfer44_mod <- freesurfer44 %>%
    select(-remove_index_col)

dim(freesurfer44_mod)


[1] 3572  353

In [7]:
for (col in 29:353){
    if(any(is.na(freesurfer44_mod[,col]))){
        
        print(col)
        print(which(is.na(freesurfer44_mod[,col])))
        print("===")
    }
}

[1] 319
[1] 52
[1] "==="


There are some rows with NAs.

instead of removing columns, we should consider removing rows.

In [8]:
freesurfer44_mod[1,1:29]

,ORIGPROT,RID,VISCODE,EXAMDATE,VERSION,FLDSTRENG,LONISID,LONIUID,IMAGEUID,RUNDATE,⋯,OVERALLQC,TEMPQC,FRONTQC,PARQC,INSULAQC,OCCQC,BGQC,CWMQC,VENTQC,ST101SV
,<fct>,<dbl>,<labelled>,<date>,<date>,<date>,<labelled>,<labelled>,<labelled>,<date>,⋯,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>
4,ADNI1,3,sc,2005-09-01,2009-07-01,NA,1948,9127,32237,2009-07-01,⋯,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,1661


In [9]:
freesurfer44_mod <- freesurfer44_mod[-c(52), ] %>% # remove row 52
    select(-c(1,4:28))
    

In [10]:
dim(freesurfer44_mod)

[1] 3571  327

In [11]:
freesurfer44_mod[1,]

,RID,VISCODE,ST101SV,ST102CV,ST102SA,ST102TA,ST102TS,ST103CV,ST103SA,ST103TA,⋯,ST97TS,ST98CV,ST98SA,ST98TA,ST98TS,ST99CV,ST99SA,ST99TA,ST99TS,ST9SV
,<dbl>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,⋯,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>
4,3,sc,1661,3261,1683,1.912,0.638,1936,722,2.287,⋯,0.608,4046,1947,2.021,0.745,10097,3292,2.579,0.659,2863


Change the data type of column between 3 and 327

In [12]:
for(col in 3:327){
    freesurfer44_mod[,col] <- as.numeric(freesurfer44_mod[,col])
}

In [13]:
freesurfer44_mod[1,]

,RID,VISCODE,ST101SV,ST102CV,ST102SA,ST102TA,ST102TS,ST103CV,ST103SA,ST103TA,⋯,ST97TS,ST98CV,ST98SA,ST98TA,ST98TS,ST99CV,ST99SA,ST99TA,ST99TS,ST9SV
,<dbl>,<labelled>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,3,sc,1661,3261,1683,1.912,0.638,1936,722,2.287,⋯,0.608,4046,1947,2.021,0.745,10097,3292,2.579,0.659,2863


In [14]:
freesurfer44_mod$VISCODE <- as.factor(gsub("sc", "bl", freesurfer44_mod$VISCODE))

In [15]:
summary(freesurfer44_mod$VISCODE)

bl   f m06 m12 m18 m24 m36 m48 m60 
773  10 728 678 303 539 341 170  29

### processing adnimerge table

The process is similar to the process of freesurfer table.

In [16]:
summary(as.factor(adnimerge$VISCODE))

bl   m0  m03  m06 m102 m108 m114  m12 m120 m126 m132 m144 m156  m18  m24  m30 
2217    1  793 1618    7  123    2 1735   81    5   62   69   34 1292 1345  750 
 m36  m42  m48  m54  m60  m66  m72  m78  m84  m90  m96   y1 
 853  308  716  201  478  217  485  213  380  129  214    1

select the diagnosis and AGE, GENDER, EDUCATION LEVEL and APOE4 and cognition scores.

there are total 3558 entries.

- AV45 has 3483 NAs. (only 75 entries)
- FDG has 1869 NAs.
- ABETA, TAU, PTAU has 2665 NAs. 

In [17]:
adnimerge_mod <- adnimerge %>%
    select(c(RID, VISCODE, DX, AGE, PTGENDER, PTEDUCAT, APOE4, CDRSB, ADAS11, ADAS13, MMSE, RAVLT.immediate, M)) 


join the freesurfer table with the adnimerge according to the `RID` and `VISCODE` columns

In [18]:
test_merge <- merge(adnimerge_mod, freesurfer44_mod, by = c("RID","VISCODE"))

In [19]:
dim(test_merge)

[1] 3558  338

In [20]:
sum(is.na(test_merge$RAVLT.immediate))

[1] 33

process the 6, 12, 18 month diagnosis outcome. 

In [21]:
"""
*CHANGE: 
Here I get the later diagnosis of all entries and mutate the value to a new column. 
I'll change the structure here to only include baseline, m06, 012, 018 entries.
"""

test_merge.bl <- test_merge[test_merge$DX %in% c("CN","Dementia"), ] %>%
    na.omit()

test_merge.M06 <- test_merge[test_merge$DX %in% c("CN","Dementia"), ] %>%    
    mutate( DX.6 = 
            apply( cbind(RID,M) ,1, function(x) getLaterDx(test_merge,x,6) ) 
          ) %>%
    na.omit()
test_merge.M06 <- test_merge.M06[!is.na(test_merge.M06$DX.6), ]

test_merge.M12 <- test_merge[test_merge$DX %in% c("CN","Dementia"), ] %>%    
    mutate( DX.12 = 
            apply( cbind(RID,M) ,1, function(x) getLaterDx(test_merge,x,12) ) 
          ) %>%
    na.omit()
test_merge.M12 <- test_merge.M12[!is.na(test_merge.M12$DX.12), ]
           
test_merge.M18 <- test_merge[test_merge$DX %in% c("CN","Dementia"), ]  
test_merge.M18 <- mutate(test_merge.M18, DX.18 = 
            apply( cbind(RID,M) ,1, function(x) getLaterDx(test_merge.M18,x,18) ) 
          ) %>%
    na.omit()
test_merge.M18 <- test_merge.M18[!is.na(test_merge.M18$DX.18), ]

In [22]:
dim(test_merge.M06)
dim(test_merge.M12)
dim(test_merge.M18)

[1] 852 339

[1] 973 339

[1] 325 339

# Predict Diagnosis Class

## Predict Diagnosis Class on 6,12,18 month later

In [ ]:
"""
*CHANGE: 
Here I should plot some figures to show the result, and then include those important variables to train a glm model. 
"""

In [23]:
cctrl1 <- trainControl(method="cv", number=10, returnResamp="all",classProbs=TRUE, summaryFunction=twoClassSummary) 

In [24]:
test_merge.M06 <- test_merge.M06 %>%
    select(-c(RID, VISCODE, DX)) %>%
    transform(DX.6 = unlist(DX.6))
test_merge.M06 <- test_merge.M06[test_merge.M06$DX.6 != "MCI",]
test_merge.M06$DX.6 <- factor(test_merge.M06$DX.6)
test_merge.M06[1:5,]

Warning message in `[<-.data.frame`(`*tmp*`, inx[matched], value = list(DX.6 = structure(c(3L, :
“replacement element 1 has 857 rows to replace 852 rows”

,AGE,PTGENDER,PTEDUCAT,APOE4,CDRSB,ADAS11,ADAS13,MMSE,RAVLT.immediate,M,⋯,ST98CV,ST98SA,ST98TA,ST98TS,ST99CV,ST99SA,ST99TA,ST99TS,ST9SV,DX.6
,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,73.9,Female,12,1,5.0,12.33,24.33,24,20,0,⋯,2999,1341,2.122,0.532,10626,3535,2.620,0.613,963,Dementia
2,73.9,Female,12,1,2.5,14.33,24.33,26,20,6,⋯,3073,1356,2.161,0.506,10045,3497,2.517,0.684,1021,Dementia
5,69.3,Male,14,1,4.5,26.67,38.67,22,19,0,⋯,3197,1430,2.081,0.823,8220,3247,2.306,0.666,2259,Dementia
6,69.3,Male,14,1,8.0,31.00,42.00,17,13,6,⋯,2879,1467,1.858,0.709,7974,3343,2.206,0.611,2372,Dementia
8,76.3,Female,18,0,0.0,4.33,9.33,30,41,0,⋯,4610,2059,2.120,0.689,13199,4061,2.790,0.772,893,CN


In [25]:
set.seed(849)
m06_cv_model <- train(DX.6 ~ . , data = test_merge.M06, method = "glmnet", trControl = cctrl1,metric = "ROC",
                             tuneGrid = expand.grid(alpha = 1,lambda = seq(0.001,0.1,by = 0.001)))

In [26]:
m06_cv_model

glmnet 

843 samples
335 predictors
  2 classes: 'CN', 'Dementia' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 759, 758, 759, 758, 759, 759, ... 
Resampling results across tuning parameters:

  lambda  ROC        Sens       Spec     
  0.001   0.7962954  0.7259136  0.7476190
  0.002   0.8152639  0.7519934  0.7619048
  0.003   0.8228919  0.7543743  0.7642857
  0.004   0.8241998  0.7495570  0.7595238
  0.005   0.8211161  0.7589701  0.7642857
  0.006   0.8198782  0.7732558  0.7690476
  0.007   0.8192651  0.7803987  0.7642857
  0.008   0.8189382  0.7874862  0.7690476
  0.009   0.8179112  0.7851606  0.7666667
  0.010   0.8171716  0.7921927  0.7690476
  0.011   0.8161631  0.7992802  0.7690476
  0.012   0.8153061  0.8016611  0.7666667
  0.013   0.8131058  0.7968992  0.7666667
  0.014   0.8104414  0.7944629  0.7690476
  0.015   0.8100894  0.8014950  0.7690476
  0.016   0.8099813  0.8038206  0.7666667
  0.017   0.8101487  0.7991141  0.7666667
  0.018   0.8

In [27]:
test_merge.M12 <- test_merge.M12 %>%
    select(-c(RID, VISCODE, DX)) %>%
    transform(DX.12 = unlist(DX.12))
test_merge.M12 <- test_merge.M12[test_merge.M12$DX.12 != "MCI",]
test_merge.M12$DX.12 <- factor(test_merge.M12$DX.12)
test_merge.M12[1:5,]

Warning message in `[<-.data.frame`(`*tmp*`, inx[matched], value = list(DX.12 = structure(c(3L, :
“replacement element 1 has 983 rows to replace 973 rows”

,AGE,PTGENDER,PTEDUCAT,APOE4,CDRSB,ADAS11,ADAS13,MMSE,RAVLT.immediate,M,⋯,ST98CV,ST98SA,ST98TA,ST98TS,ST99CV,ST99SA,ST99TA,ST99TS,ST9SV,DX.12
,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,73.9,Female,12,1,5.0,12.33,24.33,24,20,0,⋯,2999,1341,2.122,0.532,10626,3535,2.620,0.613,963,Dementia
3,73.9,Female,12,1,4.0,23.33,35.33,25,21,12,⋯,2938,1316,2.113,0.565,10033,3593,2.479,0.665,1046,Dementia
5,69.3,Male,14,1,4.5,26.67,38.67,22,19,0,⋯,3197,1430,2.081,0.823,8220,3247,2.306,0.666,2259,Dementia
8,76.3,Female,18,0,0.0,4.33,9.33,30,41,0,⋯,4610,2059,2.120,0.689,13199,4061,2.790,0.772,893,CN
10,76.3,Female,18,0,0.0,4.33,9.33,29,34,12,⋯,4599,2050,2.150,0.752,12979,3996,2.741,0.793,877,CN


In [28]:
m12_cv_model <- train(DX.12 ~ . , data = test_merge.M12, method = "glmnet", trControl = cctrl1,metric = "ROC",
                             tuneGrid = expand.grid(alpha = 1,lambda = seq(0.001,0.1,by = 0.001)))

In [29]:
m12_cv_model

glmnet 

951 samples
335 predictors
  2 classes: 'CN', 'Dementia' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 855, 856, 856, 856, 857, 856, ... 
Resampling results across tuning parameters:

  lambda  ROC        Sens       Spec       
  0.001   0.6806323  0.7308897  0.563360324
  0.002   0.6897585  0.7326754  0.571322537
  0.003   0.6923343  0.7362469  0.547638327
  0.004   0.6895702  0.7433271  0.531983806
  0.005   0.6858850  0.7573622  0.516194332
  0.006   0.6824650  0.7714599  0.484615385
  0.007   0.6808709  0.7750000  0.466396761
  0.008   0.6776963  0.7785088  0.445344130
  0.009   0.6745011  0.7925439  0.421794872
  0.010   0.6733465  0.7977757  0.392982456
  0.011   0.6726473  0.8083333  0.374561404
  0.012   0.6735252  0.8135965  0.353778677
  0.013   0.6722972  0.8171053  0.335492578
  0.014   0.6702457  0.8259085  0.324966262
  0.015   0.6692002  0.8347431  0.322334683
  0.016   0.6659739  0.8347431  0.314507422
  0.017   0.6627006  

In [30]:
test_merge.M18 <- test_merge.M18 %>%
    select(-c(RID, VISCODE, DX)) %>%
    transform(DX.18 = unlist(DX.18))
test_merge.M18 <- test_merge.M18[test_merge.M18$DX.18 != "MCI",]
test_merge.M18$DX.18 <- factor(test_merge.M18$DX.18)
test_merge.M18[1:5,]

Warning message in `[<-.data.frame`(`*tmp*`, inx[matched], value = list(DX.18 = structure(c(3L, :
“replacement element 1 has 328 rows to replace 325 rows”

,AGE,PTGENDER,PTEDUCAT,APOE4,CDRSB,ADAS11,ADAS13,MMSE,RAVLT.immediate,M,⋯,ST98CV,ST98SA,ST98TA,ST98TS,ST99CV,ST99SA,ST99TA,ST99TS,ST9SV,DX.18
,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
2,73.9,Female,12,1,2.5,14.33,24.33,26,20,6,⋯,3073,1356,2.161,0.506,10045,3497,2.517,0.684,1021,Dementia
9,76.3,Female,18,0,0.0,3.67,4.67,30,45,6,⋯,4683,2028,2.177,0.701,13154,3979,2.771,0.774,862,CN
19,75.8,Male,16,2,2.0,7.67,10.67,25,42,6,⋯,4850,1924,2.328,0.733,11746,3431,2.817,0.680,1386,CN
34,78.3,Female,13,0,0.0,8.00,14.00,27,42,6,⋯,4269,1708,2.447,0.663,11500,3601,2.754,0.600,1310,CN
40,70.7,Female,13,0,2.0,12.33,18.33,27,23,6,⋯,3679,1414,2.299,0.770,9605,2864,2.804,0.663,2032,Dementia


In [31]:
m18_cv_model <- train(DX.18 ~ . , data = test_merge.M18, method = "glmnet", trControl = cctrl1,metric = "ROC",
                             tuneGrid = expand.grid(alpha = 1,lambda = seq(0.001,0.1,by = 0.001)))

In [32]:
m18_cv_model

glmnet 

325 samples
335 predictors
  2 classes: 'CN', 'Dementia' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 293, 292, 292, 293, 292, 292, ... 
Resampling results across tuning parameters:

  lambda  ROC        Sens       Spec     
  0.001   0.5980454  0.6460784  0.5357143
  0.002   0.5980454  0.6460784  0.5357143
  0.003   0.6091099  0.6516340  0.5361905
  0.004   0.6214083  0.6516340  0.5500000
  0.005   0.6295191  0.6578431  0.5433333
  0.006   0.6409975  0.6581699  0.5500000
  0.007   0.6559539  0.6696078  0.5638095
  0.008   0.6624774  0.6692810  0.5504762
  0.009   0.6713321  0.6918301  0.5504762
  0.010   0.6793557  0.6973856  0.5571429
  0.011   0.6880626  0.7143791  0.5771429
  0.012   0.6983022  0.7143791  0.5904762
  0.013   0.7096623  0.7088235  0.5971429
  0.014   0.7152661  0.7199346  0.5971429
  0.015   0.7213414  0.7313725  0.6038095
  0.016   0.7270106  0.7254902  0.6171429
  0.017   0.7327062  0.7369281  0.6238095
  0.018   0.7

# Predict Abeta

In [ ]:
"""
*CHANGE: 
- Here I should plot some figures to show the result, and then include those 
    important variables to train a glm model. 
- Here I should use a `scale` and `center` function to pre-process data,
    preProcess = c(scale,cente)
    should consider enlarge the range of lambda.
- Consider using Boruta algorithm to find out the important variables.

"""

In [53]:
abeta_prediction <- merge(test_merge, adnimerge[,c("ABETA", "RID", "VISCODE")], by = c("RID", "VISCODE")) %>%
    select( -c("RID", "VISCODE", "DX", M)) %>%
    na.omit()

abeta_prediction$ABETA <- as.numeric(gsub("[><]", "", abeta_prediction$ABETA))
abeta_prediction <- abeta_prediction[!is.na(abeta_prediction$ABETA),]

In [57]:
cctrl1 <- trainControl(method="cv", number=10, returnResamp="all") 

abeta_cv_model <- train(ABETA ~ . , data = abeta_prediction, method = "glmnet", trControl = cctrl1,
                             tuneGrid = expand.grid(alpha = 1,lambda = seq(0.001,0.1,by = 0.001)))

In [58]:
abeta_cv_model

glmnet 

869 samples
334 predictors

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 783, 781, 781, 783, 783, 781, ... 
Resampling results across tuning parameters:

  lambda  RMSE      Rsquared   MAE     
  0.001   357.3368  0.4459651  272.8315
  0.002   357.3368  0.4459651  272.8315
  0.003   357.3368  0.4459651  272.8315
  0.004   357.3368  0.4459651  272.8315
  0.005   357.3368  0.4459651  272.8315
  0.006   357.3368  0.4459651  272.8315
  0.007   357.3368  0.4459651  272.8315
  0.008   357.3368  0.4459651  272.8315
  0.009   357.3368  0.4459651  272.8315
  0.010   357.3368  0.4459651  272.8315
  0.011   357.3368  0.4459651  272.8315
  0.012   357.3368  0.4459651  272.8315
  0.013   357.3368  0.4459651  272.8315
  0.014   357.3368  0.4459651  272.8315
  0.015   357.3368  0.4459651  272.8315
  0.016   357.3368  0.4459651  272.8315
  0.017   357.3368  0.4459651  272.8315
  0.018   357.3368  0.4459651  272.8315
  0.019   357.3368  0.4459651  272.8315


In [61]:
abeta_cv_model[["results"]]

alpha,lambda,RMSE,Rsquared,MAE,RMSESD,RsquaredSD,MAESD
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.001,357.3368,0.4459651,272.8315,35.62944,0.08728948,20.77377
1,0.002,357.3368,0.4459651,272.8315,35.62944,0.08728948,20.77377
1,0.003,357.3368,0.4459651,272.8315,35.62944,0.08728948,20.77377
1,0.004,357.3368,0.4459651,272.8315,35.62944,0.08728948,20.77377
1,0.005,357.3368,0.4459651,272.8315,35.62944,0.08728948,20.77377
1,0.006,357.3368,0.4459651,272.8315,35.62944,0.08728948,20.77377
1,0.007,357.3368,0.4459651,272.8315,35.62944,0.08728948,20.77377
1,0.008,357.3368,0.4459651,272.8315,35.62944,0.08728948,20.77377
1,0.009,357.3368,0.4459651,272.8315,35.62944,0.08728948,20.77377
